In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain

ProjDir = rel_path("..", "scripts", "12")

d = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(d) # Should be 10x5

(10, 5)

New col logpop, set log() for population data

In [2]:
d[:logpop] = map((x) -> log(x), d[:population]);
d[:society] = 1:10;

first(d, 5)

m12_6_1 = "
data{
    int total_tools[10];
    real logpop[10];
    int society[10];
}
parameters{
    real a;
    real bp;
    vector[10] a_society;
    real<lower=0> sigma_society;
}
model{
    vector[10] mu;
    sigma_society ~ cauchy( 0 , 1 );
    a_society ~ normal( 0 , sigma_society );
    bp ~ normal( 0 , 1 );
    a ~ normal( 0 , 10 );
    for ( i in 1:10 ) {
        mu[i] = a + a_society[society[i]] + bp * logpop[i];
        mu[i] = exp(mu[i]);
    }
    total_tools ~ poisson( mu );
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [3]:
stanmodel = Stanmodel(name="m12.6.1",  model=m12_6_1, num_warmup=2000,
num_samples=2000, output_format=:mcmcchain);

Input data for cmdstan

In [4]:
m12_6_1_data = Dict("total_tools" => d[:total_tools], "logpop" => d[:logpop], "society" => d[:society]);

Sample using cmdstan

In [5]:
rc, chn, cnames = stan(stanmodel, m12_6_1_data, ProjDir, diagnostics=false, CmdStanDir=CMDSTAN_HOME);


Inference for Stan model: m12_6_1_model
4 chains: each with iter=(2000,2000,2000,2000); warmup=(0,0,0,0); thin=(1,1,1,1); 8000 iterations saved.

Warmup took (0.95, 1.0, 0.93, 1.00) seconds, 3.9 seconds total
Sampling took (1.1, 1.2, 1.1, 1.1) seconds, 4.5 seconds total

                    Mean     MCSE   StdDev        5%       50%       95%    N_Eff  N_Eff/s    R_hat
lp__             9.3e+02  7.3e-02  3.1e+00   9.2e+02   9.3e+02   9.3e+02  1.8e+03  4.1e+02  1.0e+00
accept_stat__    9.5e-01  8.1e-04  7.6e-02   7.9e-01   9.8e-01   1.0e+00  8.8e+03  2.0e+03  1.0e+00
stepsize__       4.9e-02  3.1e-03  4.4e-03   4.4e-02   5.3e-02   5.3e-02  2.0e+00  4.5e-01  4.2e+13
treedepth__      5.9e+00  2.3e-02  4.7e-01   5.0e+00   6.0e+00   7.0e+00  4.1e+02  9.2e+01  1.0e+00
n_leapfrog__     7.7e+01  2.6e+00  3.9e+01   3.1e+01   6.3e+01   1.3e+02  2.3e+02  5.2e+01  1.0e+00
divergent__      0.0e+00      nan  0.0e+00   0.0e+00   0.0e+00   0.0e+00      nan      nan      nan
energy__        -9.2e+02  9

Describe the draws

In [6]:
describe(chn)

Iterations = 1:2000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 2000

Empirical Posterior Estimates:
                   Mean          SD         Naive SE         MCSE         ESS   
         lp__  927.81165188  3.093819455 0.034589953058 0.07255072720 1818.47105
accept_stat__    0.94564937  0.075504790 0.000844169213 0.00087858389 2000.00000
   stepsize__    0.04871415  0.004388071 0.000049060128 0.00049366573   79.00988
  treedepth__    5.93887500  0.474780909 0.005308211934 0.01248035640 1447.21322
 n_leapfrog__   77.17300000 39.270075609 0.439052792719 1.06744241240 1353.42616
  divergent__    0.00000000  0.000000000 0.000000000000 0.00000000000        NaN
     energy__ -921.32832700  4.049520377 0.045275014196 0.09047214455 2000.00000
            a    1.09441127  0.783710896 0.008762154191 0.01866513999 1762.98764
           bp    0.26121619  0.085513088 0.000956065385 0.00208400540 1683.71080
  a_society.1   -0.20425710  0.249787513 0.002792709299 0.00399606511 2000

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*